In [1]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
import pandas as pd

In [2]:

model = BertForSequenceClassification.from_pretrained("fine_tuned_model", num_labels=4)
tokenizer = BertTokenizer.from_pretrained("fine_tuned_model")

device = torch.device("cpu")
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [4]:
data = pd.read_csv('all_data.csv',encoding = 'cp949')
data

,comment,label
0,좌배 까는건 ㅇㅂ,0
1,개소리야 니가 빨갱이를 옹호하고 드루킹을 ㅇㅇ짓이라고 말못해서 삐진거야 빨갱아,0
2,애새끼가 초딩도 아니고 ㅋㅋㅋㅋ,0
3,731부대의 후예라 그런지 가학적인 아이디어는 세계최고임 이래서 애교만 떨어도 돈 ...,0
4,재앙이한건햇노,2
...,...,...
20170,께롱께롱께롱!!!,3
20171,갱상도가아니라 홍어지 개좃같은 홍어년들,2
20172,말레이시아랑 인도네시아 여자 존나 못생겼던데,1
20173,링크도 안박고 가서 글 쓰자고 선동하네,0


In [5]:
data['pred_0'] = None
data['pred_1'] = None
data['pred_2'] = None
data['pred_3'] = None

data

,comment,label,pred_0,pred_1,pred_2,pred_3
0,좌배 까는건 ㅇㅂ,0,None,None,None,None
1,개소리야 니가 빨갱이를 옹호하고 드루킹을 ㅇㅇ짓이라고 말못해서 삐진거야 빨갱아,0,None,None,None,None
2,애새끼가 초딩도 아니고 ㅋㅋㅋㅋ,0,None,None,None,None
3,731부대의 후예라 그런지 가학적인 아이디어는 세계최고임 이래서 애교만 떨어도 돈 ...,0,None,None,None,None
4,재앙이한건햇노,2,None,None,None,None
...,...,...,...,...,...,...
20170,께롱께롱께롱!!!,3,None,None,None,None
20171,갱상도가아니라 홍어지 개좃같은 홍어년들,2,None,None,None,None
20172,말레이시아랑 인도네시아 여자 존나 못생겼던데,1,None,None,None,None
20173,링크도 안박고 가서 글 쓰자고 선동하네,0,None,None,None,None


In [9]:
for idx, row in data.iterrows():
    sentence = row['comment']
    encoded_input = tokenizer.encode_plus(
        sentence,
        add_special_tokens=True,
        padding='max_length',
        max_length=128,
        truncation=True,
        return_tensors='pt'
    )
    input_ids = encoded_input['input_ids'].to(device)
    attention_mask = encoded_input['attention_mask'].to(device)

    
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        #predicted_labels = torch.argmax(logits, dim=1)
    data.iat[idx,2] = logits[0][0].item()
    data.iat[idx,3] = logits[0][1].item()
    data.iat[idx,4] = logits[0][2].item()
    data.iat[idx,5] = logits[0][3].item()
    if(idx%1000 == 0):
        print(idx)
    

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000


In [11]:
data[:10]

,comment,label,pred_0,pred_1,pred_2,pred_3
0,좌배 까는건 ㅇㅂ,0,1.115909,-1.194415,-0.70551,-0.159086
1,개소리야 니가 빨갱이를 옹호하고 드루킹을 ㅇㅇ짓이라고 말못해서 삐진거야 빨갱아,0,1.960328,-0.881705,-1.138325,-0.99673
2,애새끼가 초딩도 아니고 ㅋㅋㅋㅋ,0,0.573718,-1.521902,-0.596248,0.55388
3,731부대의 후예라 그런지 가학적인 아이디어는 세계최고임 이래서 애교만 떨어도 돈 ...,0,-1.715724,2.787294,2.190274,-3.055768
4,재앙이한건햇노,2,0.743082,-1.248473,-1.088047,0.585172
5,글쓴이 와꾸 승리에 비하면 방사능 피폭 원숭이 일듯..,0,3.35322,0.077062,-2.356662,-2.026903
6,마 씨발련 아 몇평이고 맷개드갔노 니 대하이햄하고 해밨나,0,3.269146,-1.141321,-0.512203,-2.743571
7,꽃다발사들고 알바하는곳찾아가서 무릎꿇고 '내 마음을 받아줄래 지영아?' 하면 바로 ...,0,1.320143,0.779342,-2.33379,-0.840387
8,박근혜 안빠는데 보수통합 3원칙 인정함,0,3.120628,-1.61859,-2.6525,-0.053392
9,대가리에 필터없는 연봉 30억 강사vs대가리가 없는 용접공,0,2.579128,-0.820354,-2.967601,0.037273


In [12]:
data.to_csv("all_data_pred.csv",encoding = 'cp949', index = False)

In [13]:
import OpenMax as om

In [14]:
ori_label = data['label']
pred_label = list(zip(data['pred_0'],data['pred_1'],data['pred_2']))
pred_label = [[a,b,c] for a,b,c in pred_label]
print(ori_label[:10])
print(pred_label[:10])

0    0
1    0
2    0
3    0
4    2
5    0
6    0
7    0
8    0
9    0
Name: label, dtype: int64
[[1.1159090995788574, -1.194414734840393, -0.7055104970932007], [1.960328459739685, -0.8817051649093628, -1.1383252143859863], [0.5737177729606628, -1.521902322769165, -0.5962476134300232], [-1.7157236337661743, 2.787294387817383, 2.1902740001678467], [0.7430822253227234, -1.2484734058380127, -1.0880465507507324], [3.353219747543335, 0.07706193625926971, -2.356661558151245], [3.26914644241333, -1.1413205862045288, -0.5122032165527344], [1.320143222808838, 0.7793415188789368, -2.3337900638580322], [3.1206283569335938, -1.618589997291565, -2.6524999141693115], [2.579127550125122, -0.8203537464141846, -2.9676005840301514]]


In [15]:
openmax = om.OpenMax()
openmax.fit(pred_label,ori_label)

In [18]:
def predict_sentence(sentence):
    encoded_input = tokenizer.encode_plus(
        sentence,
        add_special_tokens=True,
        padding='max_length',
        max_length=128,
        truncation=True,
        return_tensors='pt'
    )
    input_ids = encoded_input['input_ids'].to(device)
    attention_mask = encoded_input['attention_mask'].to(device)
    
    
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
    logits = [0,0,0,0]
    logits[0] = outputs.logits[0][0].item()
    logits[1] = outputs.logits[0][1].item()
    logits[2] = outputs.logits[0][2].item()
    logits[3] = outputs.logits[0][3].item()
    
    om_result = openmax.calculate(logits)
    _class = ['혐오','성차별','일베']
    print("입력문장 : ",sentence)
    print()
    print("모델 예측 결과 - OpenMax 미적용")
    print("혐오 : ",logits[0])
    print("성차별 : ",logits[1])
    print("일베 : ",logits[2])
    print("Unkown : ",logits[3])
    
    print()
    print("모델 예측 결과 - OpenMax 적용")
    print("Unkown : ",om_result[0])
    print("혐오 : ",om_result[1])
    print("성차별 : ",om_result[2])
    print("일베 : ",om_result[3])
    
   

In [26]:
predict_sentence("집가서 밥먹고 싶다")

입력문장 :  집가서 밥먹고 싶다

모델 예측 결과 - OpenMax 미적용
혐오 :  1.2906767129898071
성차별 :  -2.3487167358398438
일베 :  -0.10904151946306229
Unkown :  0.20658494532108307

모델 예측 결과 - OpenMax 적용
Unkown :  0.17806287140351323
혐오 :  0.6454190441459677
성차별 :  0.01700392882151623
일베 :  0.15951415562900279


In [25]:
openmax.calculate([1,1,1])

[0.10942076068263429,
 0.29677420581359937,
 0.2968040280773851,
 0.2970010054263813]